## Unsupervised Learning with Autoencoder

- Using Pytorch(macos)
- device : MPS

In [3]:
import random
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

/Users/dangchan/miniforge3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Device Setting
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps')
device

device(type='mps')

In [7]:
# Hyperparameters
epochs = 100
lr = 0.01
batch_size = 256

In [12]:
# Seed setting

def def_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def_seed(seed=123)

In [14]:
# CSV

df_train = pd.read_csv('dataset/train.csv', index_col=0)
df_val = pd.read_csv('dataset/val.csv', index_col=0)

In [15]:
class MyDataset(Dataset):
    def __init__(self, df, if_val):
        self.df = df
        self.if_val = if_val
        if self.if_val:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values

    def __getitem__(self, index):
        if self.if_val:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
    
    def __len__(self):
        return len(self.df)

In [16]:
train = MyDataset(df_train, if_val=False)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

val = MyDataset(df_val, if_val=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False)

In [19]:
# AutoEncoder

class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.Encoder = nn.Sequential(
            nn.Linear(30,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
        )
        self.Decoder = nn.Sequential(
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64,30),
        )
        
    def forward(self, x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        return x

In [21]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        self.criterion = nn.L1Loss().to(self.device)

    def fit(self, ):
        self.model.to(self.device)
        best_score = 0
        for epoch in range(epochs):
            self.model.train()
            train_loss = []
            for x in iter(self.train_loader): # for each batch
                x = x.float().to(self.device)
                self.optimizer.zero_grad() # gradient init
                _x = self.model(x) 
                loss = self.criterion(x, _x)

                loss.backward()
                self.optimizer.step()

                train_loss.append(loss.item())

            score = self.validation(self.model, 0.95)
            print(f"EPOCH {epoch}/{epochs} : Train loss : [{np.mean(train_loss)}] Val_Score : [{score}]")

            if self.scheduler is not None:
                self.scheduler.step(score)

            if best_score < score:
                best_score = score
                torch.save(model.module.state_dict(), 'best_model.pth', _use_new_zipfile_serialization=False)
            
    def validation(self, eval_model, threshold):
        cossim = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred = []
        true = []
        with torch.no_grad(): # with no backward pass(e.g. validating)
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)
                _x = self.model(x)
                cos_diff = cossim(x, _x).cpu().tolist()
                batch_pred = np.where(np.array(cos_diff)<threshold, 1.0)
                pred += batch_pred
                true += y.tolist()
        
        return f1_score(true, pred, average = 'macro')